## フェーズ1: 盤石な基盤モデルの構築 - 信頼できる評価系の確立（交差検証）(`feature/k_fold_cv`)

データパイプラインとシンプルなCNNモデルの動作確認が完了したため、次にモデルの汎化性能を客観的かつ信頼性高く評価するための基盤を構築しました。これにより、今後のモデル改善の効果を正確に測定できるようになります。

### やったこと

-   **`src/dataset.py` の改修**:
    -   `HumanCharacterDataset` クラスを汎用化し、初期化時に`image_paths`と`labels`のリストを直接受け取るように変更しました。これにより、データセットの分割ロジックを外部（`train.py`）で柔軟に制御できるようになりました。
    -   データセット内の全ての画像パスと対応するラベル、およびクラス名とインデックスのマッピングを一括で読み込む `load_all_data` 関数を新たに実装しました。

-   **`src/train.py` への交差検証の導入**:
    -   `sklearn.model_selection.StratifiedKFold` を利用し、データセット全体をK分割（今回は5分割）して層化抽出（各クラスの比率を保ったまま分割）を行いました。
    -   各フォールドにおいて、訓練データと検証データを動的に生成し、それぞれの `DataLoader` を設定しました。
    -   各フォールドごとに新しいモデルを初期化し、訓練と検証ループを実行する `train_model` 関数を定義しました。
    -   各フォールドの最終的な検証精度を記録し、全フォールドの平均検証精度と標準偏差を算出・表示することで、モデル性能のよりロバストな評価を可能にしました。

### 動作確認

構築した交差検証を含む学習スクリプトは、`src/train.py` を直接実行することで動作確認が可能です。

```bash
# srcディレクトリ内から実行
python3 train.py
```
実行すると、各フォールドの訓練・検証の進捗、エポックごとの損失と精度、そして最終的に全フォールドの平均検証精度と標準偏差が出力されます。

例:
```
--- Fold 1/5 ---
  訓練データ数 (Fold 1): 2000
  検証データ数 (Fold 1): 501
...
Fold 1 Final Validation Accuracy: 49.10%

--- Fold 2/5 ---
...
Fold 2 Final Validation Accuracy: 52.60%

...

--- K-Fold Cross-Validation 完了 ---
各フォールドの検証精度: [49.10179640718563, 52.6, 56.4, 53.8, 54.0]
平均検証精度: 53.18%
標準偏差: 2.38%
```